# (Optional Lab) Nova Prompt Optimizer

## Introduction

The Nova Prompt Optimizer is a powerful tool that automatically improves your prompts for Amazon Nova models using your own datasets. This workshop demonstrates how to:

1. Transform manual prompt engineering into an efficient, data-driven process
2. Optimize prompts specifically tailored to your use case and data
3. Evaluate the performance improvements from optimization

By the end of this notebook, you'll understand how to leverage automated prompt optimization to unlock the full potential of Amazon Nova models for your specific applications.

<div class="alert alert-block alert-warning">
    <b>Optional Lab</b> 
    
    This notebook takes 15 mins to run. Recommend to treat it as an optional notebook for AWS hosted event.
</div>

## Section 1: Setup and Installation

We'll start by installing the Nova Prompt Optimizer SDK, which provides the tools needed to automatically optimize prompts based on your data.

In [19]:
import sys
!{sys.executable} -m pip install nova-prompt-optimizer

  Using cached numpy-2.3.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached numpy-2.3.1-cp312-cp312-manylinux_2_28_x86_64.whl (16.6 MB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2━━━━━━ 0/2 [numpy]
  Attempting uninstall: dill━━━━━━━━━━━━━━━━━━━━ 0/2 [numpy]
    Found existing installation: dill None━━ 0/2 [numpy]
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [dill]error: uninstall-no-record-file

× Cannot uninstall dill None
╰─> The package's contents are unknown: no RECORD file was found for dill.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps dill==0.4.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [dill]


## Section 2: Initialize the Input Adapters

The Nova Prompt Optimizer uses adapters to standardize inputs from different sources. These adapters help connect your data, prompts, and evaluation metrics into the optimization pipeline.

![adapters](nova_prompt_optimizer/docs/adapters.png)

### 2.1 Dataset Adapter

The Dataset Adapter converts your data into a standardized format for optimization and evaluation:

- **Input Columns**: Specify which fields from your data will be used as inputs to the model
- **Output Columns**: Specify which fields contain the expected outputs for comparison
- **Train/Test Split**: Divide your dataset for optimization and evaluation

In [20]:
from amzn_nova_prompt_optimizer.core.input_adapters.dataset_adapter import JSONDatasetAdapter

# Define which columns in our dataset contain inputs and expected outputs
input_columns = {"input"}  # The field containing the user's query
output_columns = {"answer"}  # The field containing the expected model response

# Initialize the dataset adapter for our JSONL dataset
dataset_adapter = JSONDatasetAdapter(input_columns, output_columns)

# Load and process the dataset
dataset_adapter.adapt("nova_prompt_optimizer/data/FacilitySupportAnalyzer.jsonl")

# Split into training data (for optimization) and test data (for evaluation)
train_set, test_set = dataset_adapter.split(0.5)  # 50/50 split

### 2.2 Prompt Adapter

The Prompt Adapter standardizes your existing prompt template:

- **Prompt Variables**: Identify placeholders in your prompt that should be replaced with data from input columns
- **File Path**: Provide the path to your original prompt template
- **Adapt**: Process the prompt into a standardized format for optimization

In [21]:
from amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter import TextPromptAdapter

# Define which variables in our prompt will be replaced with data from input columns
prompt_variables = input_columns

# Initialize the prompt adapter for a text prompt file
prompt_adapter = TextPromptAdapter()

# Load the original prompt template and specify which variables to replace
prompt_adapter.set_user_prompt(file_path="nova_prompt_optimizer/original_prompt/user_prompt_template.txt", variables=prompt_variables)

# Process the prompt into a standardized format
prompt_adapter.adapt()

2025/07/25 19:36:14 INFO amzn_nova_prompt_optimizer.core.input_adapters.prompt_adapter: System Prompt not set, initializing as empty string...


### 2.3 Metric Adapter

The Metric Adapter defines how to evaluate prompt performance:

- **Custom Metrics**: Create evaluation metrics specific to your task
- **Apply Function**: Evaluate a single model response against the expected output
- **Batch Apply Function**: Evaluate multiple responses at once

For this example, we'll create a custom metric for the Facility Support Analyzer task that measures:
1. JSON validity
2. Correctness of categories
3. Accuracy of sentiment classification
4. Accuracy of urgency classification

In [5]:
from amzn_nova_prompt_optimizer.core.input_adapters.metric_adapter import MetricAdapter
from typing import List, Any, Dict
import re
import json

class FacilitySupportAnalyzerMetric(MetricAdapter):
    def parse_json(self, input_string: str):
        """
        Attempts to parse the given string as JSON. If direct parsing fails,
        it tries to extract a JSON snippet from code blocks formatted as:
            ```json
            ... JSON content ...
            ```
        or any code block delimited by triple backticks and then parses that content.
        """
        try:
            return json.loads(input_string)
        except json.JSONDecodeError as err:
            error = err

        patterns = [
            re.compile(r"```json\s*(.*?)\s*```", re.DOTALL | re.IGNORECASE),
            re.compile(r"```(.*?)```", re.DOTALL)
        ]

        for pattern in patterns:
            match = pattern.search(input_string)
            if match:
                json_candidate = match.group(1).strip()
                try:
                    return json.loads(json_candidate)
                except json.JSONDecodeError:
                    continue

        raise error

    def _calculate_metrics(self, y_pred: Any, y_true: Any) -> Dict:
        strict_json = False
        result = {
            "is_valid_json": False,
            "correct_categories": 0.0,
            "correct_sentiment": False,
            "correct_urgency": False,
        }

        try:
            y_true = y_true if isinstance(y_true, dict) else (json.loads(y_true) if strict_json else self.parse_json(y_true))
            y_pred = y_pred if isinstance(y_pred, dict) else (json.loads(y_pred) if strict_json else self.parse_json(y_pred))
        except json.JSONDecodeError:
            result["total"] = 0
            return result  # Return result with is_valid_json = False
        else:
            result["is_valid_json"] = True

            categories_true = y_true.get("categories", {})
            categories_pred = y_pred.get("categories", {})

            if isinstance(categories_true, dict) and isinstance(categories_pred, dict):
                correct = sum(
                    categories_true.get(k, False) == categories_pred.get(k, False)
                    for k in categories_true
                )
                result["correct_categories"] = correct / len(categories_true) if categories_true else 0.0
            else:
                result["correct_categories"] = 0.0  # or raise an error if you prefer

            result["correct_sentiment"] = y_pred.get("sentiment", "") == y_true.get("sentiment", "")
            result["correct_urgency"] = y_pred.get("urgency", "") == y_true.get("urgency", "")

        # Compute overall metric score
        result["total"] = sum(
            float(result[k]) for k in ["correct_categories", "correct_sentiment", "correct_urgency"]
        ) / 3.0

        return result

    def apply(self, y_pred: Any, y_true: Any):
        return self._calculate_metrics(y_pred, y_true)

    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        evals = [self.apply(y_pred, y_true) for y_pred, y_true in zip(y_preds, y_trues)]
        float_keys = [k for k, v in evals[0].items() if isinstance(v, (int, float, bool))]
        return {k: sum(e[k] for e in evals) / len(evals) for k in float_keys}

metric_adapter = FacilitySupportAnalyzerMetric()

### 2.4 Inference Adapter

The Inference Adapter connects to the model service:

- **Backend**: Currently supports Amazon Bedrock
- **Region**: Specify which AWS region to use for inference
- **Configuration**: Set up the connection to the inference service

In [6]:
from amzn_nova_prompt_optimizer.core.inference.adapter import BedrockInferenceAdapter

# Initialize the inference adapter to connect to Amazon Bedrock
# We're using us-west-2 region for this example
inference_adapter = BedrockInferenceAdapter(region_name="us-west-2")

## Section 3: Evaluate the Original Prompt

Before optimization, we'll establish a baseline by evaluating the original prompt's performance on our test dataset. This will help us measure the improvement from optimization.

The Evaluator:
- Takes our prompt, test data, metrics, and inference adapter
- Generates predictions using the original prompt
- Calculates evaluation metrics on these predictions

#### Base Model Evaluation

In [7]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

# Initialize the evaluator with all our components
# - prompt_adapter: The prompt to evaluate
# - test_set: Data to run the evaluation on
# - metric_adapter: How to calculate performance metrics
# - inference_adapter: Connection to the model service
evaluator = Evaluator(prompt_adapter, test_set, metric_adapter, inference_adapter)

In [8]:
# Run evaluation of the original prompt with Amazon Nova Lite
# This will generate predictions and calculate metrics
original_prompt_score = evaluator.aggregate_score(model_id="us.amazon.nova-lite-v1:0")

print(f"Original Prompt Evaluation Score = {original_prompt_score}")

2025/07/25 19:12:02 INFO amzn_nova_prompt_optimizer.core.evaluation: Cache miss - Running new inference on Dataset
Running inference: 100%|██████████| 100/100 [00:39<00:00,  2.54it/s]
2025/07/25 19:12:41 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Batch Evaluation on Dataset, using `batch_apply` metric
2025/07/25 19:12:41 INFO amzn_nova_prompt_optimizer.core.evaluation: Using cached inference results
2025/07/25 19:12:41 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Evaluation on Dataset, using `apply` metric


Original Prompt Evaluation Score = {'is_valid_json': 1.0, 'correct_categories': 0.888, 'correct_sentiment': 0.57, 'correct_urgency': 0.65, 'total': 0.7026666666666667}


## Section 4: Optimize the Prompt

Now we'll use the Nova Prompt Optimizer to automatically improve our prompt based on the training data.

### 4.1 Optimization Metric

First, we need to adapt our metric for the optimizer, which requires a single numerical score instead of multiple metrics:

In [9]:
class FacilitySupportAnalyzerNovaPromptOptimizerMetric(FacilitySupportAnalyzerMetric):
    def apply(self, y_pred: Any, y_true: Any):
        """
        Returns a single numerical value for the optimizer to use.
        The optimizer needs a single score to maximize during optimization.
        
        Args:
            y_pred: The model's prediction
            y_true: The expected output
            
        Returns:
            float: A score between 0 and 1, with higher being better
        """
        # Calculate metrics and return the total score (average of all metrics)
        return self._calculate_metrics(y_pred, y_true)["total"]
        
    def batch_apply(self, y_preds: List[Any], y_trues: List[Any]):
        # Not used during optimization
        pass
    
# Create the metric adapter for optimization
nova_prompt_optimizer_metric_adapter = FacilitySupportAnalyzerNovaPromptOptimizerMetric()

### 4.2 Optimization Adapters

Next, we'll set up the optimization process. The Nova Prompt Optimizer takes:

- **Prompt Adapter**: The original prompt to optimize
- **Inference Adapter**: Connection to the model service
- **Dataset Adapter**: Training data to learn from
- **Metric Adapter**: How to evaluate prompt performance

### 4.3 Nova Prompt Optimizer

The Nova Prompt Optimizer uses a two-stage approach:

1. **Meta Prompting**: Analyzes your prompt to identify system instructions and user template patterns
2. **MIPROv2 Optimization**: Improves system instructions and adds few-shot examples based on your dataset

The optimizer can run in different modes based on your Nova model:
- **Lite mode**: Optimized for Nova Lite, faster optimization with fewer resources
- **Pro mode**: Optimized for Nova Pro, more thorough optimization that may take longer

In [10]:
from amzn_nova_prompt_optimizer.core.optimizers import NovaPromptOptimizer

# Initialize the Nova Prompt Optimizer with our components
nova_prompt_optimizer = NovaPromptOptimizer(
    prompt_adapter=prompt_adapter,        # Original prompt to optimize
    inference_adapter=inference_adapter,  # Connection to model service
    dataset_adapter=train_set,            # Training data to learn from
    metric_adapter=nova_prompt_optimizer_metric_adapter  # How to evaluate performance
)

# Run the optimization process in "lite" mode for Nova Lite
# This will analyze the prompt, identify improvements, and generate few-shot examples
optimized_prompt_adapter = nova_prompt_optimizer.optimize(mode="lite")

2025/07/25 19:12:44 INFO amzn_nova_prompt_optimizer.core.optimizers.nova_meta_prompter.nova_mp_optimizer: Optimizing prompt using Nova Meta Prompter with Model: us.amazon.nova-premier-v1:0
2025/07/25 19:12:50 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using us.amazon.nova-lite-v1:0 for Evaluation
2025/07/25 19:12:50 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using us.amazon.nova-premier-v1:0 for Prompting
2025/07/25 19:12:50 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.custom_adapters.custom_chat_adapter: Initializing CustomChatAdapter with enable_json_fallback=False
2025/07/25 19:12:50 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Using Nova tips for MIPROv2 optimization
2025/07/25 19:12:50 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/25 19:12:50 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates

Bootstrapping set 1/20
Bootstrapping set 2/20
Bootstrapping set 3/20


  0%|          | 0/50 [00:01<?, ?it/s]
2025/07/25 19:12:51 INFO dspy.teleprompt.mipro_optimizer_v2: Error generating few-shot examples: 'NoneType' object is not subscriptable
2025/07/25 19:12:51 INFO dspy.teleprompt.mipro_optimizer_v2: Running without few-shot examples.
2025/07/25 19:12:51 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Entering patched_propose_instructions, patching GroundedProposer with NovaGroundedProposer
2025/07/25 19:12:51 INFO amzn_nova_prompt_optimizer.core.optimizers.miprov2.miprov2_optimizer: Patched GroundedProposer, current GroundedProposer class=<class 'amzn_nova_prompt_optimizer.core.optimizers.nova_prompt_optimizer.nova_grounded_proposer.NovaGroundedProposer'>
2025/07/25 19:12:51 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/07/25 19:12:51 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of th

[Nova] Selected tip: structured_prompt
[Nova] Selected tip: high_stakes
[Nova] Selected tip: multi_turn
[Nova] Selected tip: examples
[Nova] Selected tip: rules_based
[Nova] Selected tip: examples
[Nova] Selected tip: none
[Nova] Selected tip: simple
[Nova] Selected tip: structured_prompt
[Nova] Selected tip: format_control
[Nova] Selected tip: description
[Nova] Selected tip: simple
[Nova] Selected tip: rules_based
[Nova] Selected tip: format_control
[Nova] Selected tip: creative
[Nova] Selected tip: description
[Nova] Selected tip: format_control
[Nova] Selected tip: high_stakes
[Nova] Selected tip: format_control
[Nova] Selected tip: none


2025/07/25 19:15:30 INFO dspy.teleprompt.mipro_optimizer_v2: Proposed Instructions for Predictor 0:

2025/07/25 19:15:30 INFO dspy.teleprompt.mipro_optimizer_v2: 0: **Task:**
Extract and return a JSON with specified keys and values based on the input.

**Context:**
- The JSON must include "urgency", "sentiment", and "categories".
- "urgency" can be `high`, `medium`, or `low`.
- "sentiment" can be `negative`, `neutral`, or `positive`.
- "categories" is a dictionary with boolean values indicating if each category matches the input.

**Instructions:**
- MUST include all specified keys: "urgency", "sentiment", and "categories".
- "categories" MUST include all listed support category tags with boolean values.
- The JSON string MUST be valid and readable directly.
- DO NOT enclose the JSON in ```json...```.
- DO NOT include newlines or unnecessary whitespaces.

**Response Format:**
- The response MUST be a single-line JSON string.
- MUST adhere to the specified format and include all require

...


Average Metric: 25.40 / 35 (72.6%): 100%|██████████| 35/35 [00:19<00:00,  1.77it/s]

2025/07/25 19:25:35 INFO dspy.evaluate.evaluate: Average Metric: 25.4 / 35 (72.6%)
2025/07/25 19:25:35 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 72.57 on minibatch of size 35 with parameters ['Predictor 0: Instruction 5'].
2025/07/25 19:25:35 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [64.38, 74.38, 69.33, 71.9, 72.1, 72.95, 70.38, 72.19, 73.52, 73.62, 73.71, 72.95, 73.52, 71.9, 74.76, 68.1, 67.43, 73.33, 70.57, 74.38, 67.81, 76.76, 75.62, 75.43, 78.0, 77.33, 76.1, 76.57, 72.76, 72.57]
2025/07/25 19:25:35 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [65.27, 74.13, 72.2, 73.13, 75.13, 74.27]
2025/07/25 19:25:35 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 75.13
2025/07/25 19:25:35 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================




2025/07/25 19:25:35 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 37 / 37 - Full Evaluation =====
2025/07/25 19:25:35 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 72.95) from minibatch trials...


Average Metric: 36.03 / 50 (72.1%): 100%|██████████| 50/50 [00:21<00:00,  2.31it/s]

2025/07/25 19:25:57 INFO dspy.evaluate.evaluate: Average Metric: 36.03333333333333 / 50 (72.1%)
2025/07/25 19:25:57 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [65.27, 74.13, 72.2, 73.13, 75.13, 74.27, 72.07]
2025/07/25 19:25:57 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 75.13
2025/07/25 19:25:57 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/07/25 19:25:57 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/07/25 19:25:57 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 75.13!


### 4.4 Examining the Optimized Prompt

Let's examine what the optimizer has produced. First, the optimized system prompt:

In [11]:
optimized_prompt_adapter.system_prompt

'Given a facility management service request, analyze the text to determine its urgency, sentiment, and relevant categories. Return a JSON object with the keys "urgency", "sentiment", and "categories". "urgency" should be one of `high`, `medium`, or `low`. "sentiment" should be `negative`, `neutral`, or `positive`. "categories" should include boolean values for "quality", "safety", and "urgent repairs". Ensure the JSON is valid, single-line, and includes all required keys with appropriate values.'

### 4.5 Optimized User Prompt

Now let's look at the optimized user prompt template:

In [12]:
optimized_prompt_adapter.user_prompt

'Extract and return a json with the following keys and values from the input provided: [{{ input }}]\n- "urgency" as one of `high`, `medium`, `low`\n- "sentiment" as one of `negative`, `neutral`, `positive`\n- "categories" as a dictionary with categories as keys and boolean values indicating if the category matches the input. Categories are: `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`, `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`, `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`, `facility_management_issues`'

### 4.6 Saving the Optimized Prompt

Let's save the optimized prompt for future use:

In [15]:
optimized_prompt_adapter.save("nova_prompt_optimizer/optimized_prompt/")

## Section 5: Evaluate the Optimized Prompt

Now let's measure the performance of our optimized prompt on the test dataset to see how much improvement we've gained:

In [16]:
from amzn_nova_prompt_optimizer.core.evaluation import Evaluator

# Create a new evaluator for the optimized prompt
evaluator = Evaluator(
    optimized_prompt_adapter,  # Now using the optimized prompt
    test_set,                  # Same test data as before
    metric_adapter,            # Same evaluation metrics
    inference_adapter          # Same model service
)

In [17]:
# Run evaluation of the optimized prompt with Amazon Nova Lite
nova_prompt_optimizer_eval_score = evaluator.aggregate_score(model_id="us.amazon.nova-lite-v1:0")

2025/07/25 19:30:28 INFO amzn_nova_prompt_optimizer.core.evaluation: Cache miss - Running new inference on Dataset
Running inference: 100%|██████████| 100/100 [00:39<00:00,  2.53it/s]
2025/07/25 19:31:08 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Batch Evaluation on Dataset, using `batch_apply` metric
2025/07/25 19:31:08 INFO amzn_nova_prompt_optimizer.core.evaluation: Using cached inference results
2025/07/25 19:31:08 INFO amzn_nova_prompt_optimizer.core.evaluation: Running Evaluation on Dataset, using `apply` metric


In [22]:
# Print the score and compare it to the original prompt
print(f"Optimized Prompt Evaluation Score = {nova_prompt_optimizer_eval_score}")
print(f"Improvement: {nova_prompt_optimizer_eval_score['total'] - original_prompt_score['total']:.4f} ({(nova_prompt_optimizer_eval_score['total'] - original_prompt_score['total']) / original_prompt_score['total'] * 100:.2f}%)")

Optimized Prompt Evaluation Score = {'is_valid_json': 1.0, 'correct_categories': 0.905, 'correct_sentiment': 0.55, 'correct_urgency': 0.91, 'total': 0.7883333333333333}
Improvement: 0.0857 (12.19%)


### 5.1 Saving Evaluation Results

Let's save the detailed evaluation results for analysis:

In [23]:
evaluator.save("nova_prompt_optimizer/evals/nova_lite/nova_prompt_optimizer_eval.jsonl")

2025/07/25 19:36:26 INFO amzn_nova_prompt_optimizer.core.evaluation: Successfully saved evaluation results to nova_prompt_optimizer/evals/nova_lite/nova_prompt_optimizer_eval.jsonl


# Conclusion

In this workshop, we've explored how to use the Nova Prompt Optimizer to automatically improve prompt performance for Amazon Nova models. Let's summarize what we've learned and the benefits of this approach.

## Key Learnings

### 1. The Power of Automated Optimization
- **Data-Driven Improvements**: Rather than manual trial-and-error, we used our own dataset to guide prompt optimization
- **Systematic Approach**: The optimizer methodically analyzes and enhances prompts through meta-prompting and few-shot learning
- **Measurable Results**: We quantitatively measured performance gains between original and optimized prompts

### 2. Components of the Nova Prompt Optimizer
- **Dataset Adapter**: Standardized our dataset for use in optimization and evaluation
- **Prompt Adapter**: Processed our original prompt into a format suitable for optimization
- **Metric Adapter**: Provided custom evaluation metrics specific to our task
- **Inference Adapter**: Connected us to Amazon Nova models for testing
- **Optimization Process**: Combined meta-prompting and MIPROv2 techniques

### 3. Optimization Techniques Applied
- **System Prompt Refinement**: Improved the system instructions for better task understanding
- **Few-Shot Example Selection**: Automatically identified the most helpful examples from our data
- **Format Optimization**: Enhanced output formatting and structure
- **Task-Specific Guidance**: Added task-specific tips and clarifications

## Benefits for Production Applications

1. **Reduced Engineering Time**: Automates the time-consuming process of prompt engineering
2. **Consistent Performance**: Creates reliable, tested prompts for production use
3. **Adaptability**: Easily update optimized prompts as your data or requirements change
4. **Model Flexibility**: Works with different Amazon Nova models (Micro, Lite, Pro)
5. **Customization**: Optimizes for your specific data and task requirements

## Next Steps

As you apply the [Nova Prompt Optimizer](https://github.com/aws/nova-prompt-optimizer) to your own projects, consider:

1. **Expand Your Dataset**: Larger, more diverse datasets often yield better optimization results
2. **Test Different Metrics**: Create custom metrics that align closely with your business goals
3. **Compare Models**: Try optimizing for different Nova models to find the best performance/cost balance
4. **Periodic Re-optimization**: Update your prompts as your data or requirements evolve
5. **Integration**: Incorporate optimized prompts into your production applications